> # **Visualização de dados Corona Vírus no Brasil**

In [ ]:
import pandas as pd
import numpy as np

import plotly.express as px
from plotly.subplots import make_subplots
from plotly.graph_objs import *
import plotly.graph_objects as go

from fbprophet import Prophet
from sklearn.metrics import mean_squared_error, mean_absolute_error

import folium
import json

import datetime

import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns

In [ ]:
path = "../input/corona-virus-brazil/brazil_covid19_macro.csv"
df = pd.read_csv(path, encoding='utf-8')

df['date'] = pd.to_datetime(df['date'])
df.head()

In [ ]:
def isweekend(date):
    #date is date type
    day_of_the_week =  date.weekday()
    if day_of_the_week < 5:
        return False 
    else:
        return True

In [ ]:
df['day_or_end'] = df.date.apply(lambda x: isweekend(x))
df = df[['date','day_or_end', 'cases', 'deaths', 'recovered']]

df.info()

df['active'] = df['cases'] - (df['recovered'] - df['deaths'])
df['new_cases'] = df.cases.diff()
df['new_deaths']= df.deaths.diff()

#Adiciona uma coluna ao DataFrame
df['mortality'] = df['deaths']*100/df['cases']
df['mortality'] = df['mortality'].fillna(0)

df['MA cases'] = df['new_cases'].rolling(window=7).mean()
df['MA deaths'] = df['new_deaths'].rolling(window=7).mean()

#Substitui NaN por 0
df = df.fillna(0)
df.tail()

In [ ]:
pathpop2 = '../input/dadosestados22222/86d279230ce221ce09ef4a5029cb46b1.csv'
df_pop2 = pd.read_csv(pathpop2, sep = ',',encoding = 'utf-8')

df_pop2 = df_pop2[['UF','capital', 'área', 'população', 'densidade_demog', 'matriculas_ef', 'IDH',
                  'receitas', 'despesas', 'total_veiculos']]

df_pop2 = df_pop2.sort_values(by = 'UF', ignore_index = True).copy()
display(df_pop2.head())

pop = df_pop2['população'].sum()
print('A população do Brasil é: %d' % (pop))

avgmort = df['mortality'].mean()/100*(pop/100000)
print('A mortalidade atual no Brasil é: %.2f para cada 100.000 habitantes' % avgmort)

In [ ]:
fig = make_subplots(rows=1, cols=2, shared_yaxes=True)

fig.add_trace(
    go.Bar(
        x=df['date'],
        y=df['cases'],
        #line=dict(color='green', width=3),
        name='Casos',
        marker_color='lightslategray'), row=1, col=1)

fig.add_trace(
    go.Bar(
        x=df['date'],
        y=df['deaths'],
        #line=dict(color='green', width=3),
        marker_color='red',
        name='Mortes'), row=1, col=1)

fig.add_trace(
    go.Bar(
        x=['Casos', 'Mortes'],
        y=[df['cases'].max(), df['deaths'].max()],
        #line=dict(color='green', width=3),
        marker_color=['lightslategray','red'],
        showlegend=False,
        text = [df['cases'].max(), df['deaths'].max()],
        texttemplate='%{text:.f}',
        textposition = 'outside'), row=1, col=2)

fig.update_layout(hovermode = 'x',
                  barmode = 'overlay',
                  title_text = 'Total de casos confirmados por dia',
                  legend = dict(orientation = 'h', yanchor="bottom",y=1.01,xanchor="right",x=1))


fig.update_xaxes(title_text="Dia", row=1, col=1)
fig.update_xaxes(title_text="Total de casos e de mortes", row=1, col=2)                
fig.update_yaxes(title_text='Quantidade',row=1, col=1)

fig.show()

In [ ]:
fign = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.02)

fign.add_trace(
    go.Bar(
        x=df['date'],
        y=df['new_cases'],
        #line=dict(color='green', width=3),
        name='Novos casos',
        marker_color='lightslategray'
    ), row = 1, col = 1
)

fign.add_trace(
    go.Scatter(
        x=df['date'],
        y=df['MA cases'],
        #line=dict(color='green', width=3),
        name='Média móvel de novos casos',
        marker_color='green'
    ), row = 1, col = 1
)

fign.add_trace(
    go.Bar(
        x=df['date'],
        y=df['new_deaths'],
        #line=dict(color='green', width=3),
        marker_color='crimson',
        name='Novas mortes'
    ), row = 2, col = 1
)

fign.add_trace(
    go.Scatter(
        x=df['date'],
        y=df['MA deaths'],
        #line=dict(color='green', width=3),
        marker_color='blue',
        name='Média móvel de novas mortes'
    ), row = 2, col = 1
)


fign.update_layout(hovermode = 'x',
                   barmode = 'overlay',
                   title_text = 'Incrementos diários de novos casos e novas mortes',
                   legend=dict(orientation = 'h', yanchor="bottom",y=1.01,xanchor="right",x=1))
fign.update_xaxes(title_text='Dia', row=2, col=1)
#fign.update_yaxes(title_text='Quantidade')
fign.show()

In [ ]:
df_day = df.loc[df['day_or_end'] == False].reset_index(drop = True)
df_day['MA cases'] = df_day['new_cases'].rolling(window=7).mean()
df_day['MA deaths'] = df_day['new_deaths'].rolling(window=7).mean()
display(df_day.tail())

df_end = df.loc[df['day_or_end'] == True].reset_index(drop = True)
df_end['MA cases'] = df_end['new_cases'].rolling(window=7).mean()
df_end['MA deaths'] = df_end['new_deaths'].rolling(window=7).mean()
display(df_end.tail())

In [ ]:
fign = go.Figure()

fign.add_trace(
    go.Bar(
        x=df_day['date'],
        y=df_day['new_cases'],
        #line=dict(color='green', width=3),
        name='Novos casos em dias úteis',
        marker_color='lightslategray'
    )
)

fign.add_trace(
    go.Bar(
        x=df_end['date'],
        y=df_end['new_cases'],
        #line=dict(color='green', width=3),
        name='Novos casos em finais de semana',
        marker_color='green'
    )
)

fign.add_trace(
    go.Scatter(
        x=df_day['date'],
        y=df_day['MA cases'],
        #line=dict(color='green', width=3),
        name='Média móvel de novos casos em dias úteis',
        marker_color='black'
    )
)

fign.add_trace(
    go.Scatter(
        x=df_end['date'],
        y=df_end['MA cases'],
        #line=dict(color='green', width=3),
        name='Média móvel de novos casos em finais de semana',
        marker_color='blue'
    )
)


fign.update_layout(hovermode = 'x',
                   barmode = 'overlay',
                   title_text = 'Incrementos diários de novos casos em dias úteis e finais de semana',
                   legend=dict(orientation = 'v', yanchor="top",y=0.98,xanchor="left",x=0.01))
fign.update_xaxes(title_text='Dia')
fign.update_yaxes(title_text='Quantidade')
fign.show()

In [ ]:
fign = go.Figure()

fign.add_trace(
    go.Bar(
        x=df_day['date'],
        y=df_day['new_deaths'],
        #line=dict(color='green', width=3),
        marker_color='mediumaquamarine',
        name='Novas mortes em dias úteis'
    )
)

fign.add_trace(
    go.Bar(
        x=df_end['date'],
        y=df_end['new_deaths'],
        #line=dict(color='green', width=3),
        marker_color='green',
        name='Novas mortes em finais de semana'
    )
)

fign.add_trace(
    go.Scatter(
        x=df_day['date'],
        y=df_day['MA deaths'],
        #line=dict(color='green', width=3),
        name='Média móvel de novas mortes em dias úteis',
        marker_color='black'
    )
)

fign.add_trace(
    go.Scatter(
        x=df_end['date'],
        y=df_end['MA deaths'],
        #line=dict(color='green', width=3),
        name='Média móvel de novas mortes em finais de semana',
        marker_color='blue'
    )
)

fign.update_layout(hovermode = 'x',
                   barmode = 'overlay',
                   title_text = 'Incrementos diários de novas mortes em dias úteis e finais de semana',
                   legend=dict(orientation = 'h', yanchor="top",y=1.15,xanchor="right",x=0.98))
fign.update_xaxes(title_text='Dia')
fign.update_yaxes(title_text='Quantidade')
fign.show()

In [ ]:
figb = make_subplots(rows=1, cols=2, shared_yaxes=True)

figb.add_trace(
    go.Bar(
        x=df['date'],
        y=df['recovered'],
        #line=dict(color='green', width=3),
        name='Recuperados',
        marker_color='#17becf'), row=1, col=1)

figb.add_trace(
    go.Bar(
        x=df['date'],
        y=df['active'],
        #line=dict(width=3),
        name='Casos Ativos',
        opacity = 0.7,
        marker_color='gray'), row=1, col=1)

figb.add_trace(
    go.Bar(
        x=df['date'],
        y=df['deaths'],
        #line=dict(width=3),
        name='Mortes',
        marker_color='#d62728'), row=1, col=1)

figb.add_trace(
    go.Bar(
        x=['Recuperados', 'Ativos', 'Mortes'],
        y=[df['recovered'].iloc[-1],df['active'].iloc[-1], df['deaths'].iloc[-1]],
        #line=dict(color='green', width=3),
        marker_color=['#17becf', 'gray', '#d62728'],
        showlegend=False,
        text = [df['recovered'].iloc[-1],df['active'].iloc[-1], df['deaths'].iloc[-1]],
        texttemplate='%{text:.f}',
        textposition = 'outside'), row=1, col=2)

figb.update_layout(hovermode = 'x',
                  barmode = 'overlay',
                  title_text = 'Recuperados, ativos e mortos, por dia e total',
                  legend = dict(orientation = 'h', yanchor="bottom",y=1.01,xanchor="right",x=1))


figb.update_xaxes(title_text="Dia", row=1, col=1)
figb.update_xaxes(title_text="Total de recuperados, ativos e mortos", row=1, col=2)                
figb.update_yaxes(title_text='Quantidade',row=1, col=1)

figb.show()

https://www.kaggle.com/dferhadi/covid-19-predictions-growth-factor-and-calculus

In [ ]:
def smoother(inputdata,w,imax):
    data = 1.0*inputdata
    data = data.replace(np.nan,1)
    data = data.replace(np.inf,1)
    #print(data)
    smoothed = 1.0*data
    normalization = 1
    for i in range(-imax,imax+1):
        if i==0:
            continue
        smoothed += (w**abs(i))*data.shift(i,axis=0)
        normalization += w**abs(i)
    smoothed /= normalization
    return smoothed

In [ ]:
dfc = df.drop(columns = ['new_cases','new_deaths','mortality'])

dfc['confirmed_iminus1'] = dfc['cases'].shift(1, axis=0)
dfc['confirmed_iminus2'] = dfc['cases'].shift(2, axis=0)

dfc = dfc.fillna(1)

#display(dfc.head(30))

dfc['growth_factor'] = (dfc['cases']-dfc['confirmed_iminus1'])/(dfc['confirmed_iminus1']-dfc['confirmed_iminus2'])
dfc['growth_ratio'] = (dfc['cases']/dfc['confirmed_iminus1'])
dfc['growth_rate'] = np.gradient(np.log(dfc['cases'] + 1e-6))
dfc['2derivative'] = np.gradient(np.gradient(dfc['cases']))

dfc = dfc.replace([np.inf, -np.inf], np.nan)
dfc = dfc.fillna(1)

#display(dfc.tail(20))

dfc['growth_factor'] = smoother(dfc['growth_factor'], 0.5, 5)
dfc['growth_ratio'] = smoother(dfc['growth_ratio'], 0.5, 5)
dfc['growth_rate'] = smoother(dfc['growth_rate'], 0.5, 3)
dfc['2derivative'] = smoother(dfc['2derivative'], 0.5, 7)

dfc = dfc.fillna(1)

#dfc.tail(20)

In [ ]:
fign = make_subplots(rows=4, cols=1, shared_xaxes=True, vertical_spacing=0.02)

fign.add_trace(
    go.Scatter(
        x=dfc['date'],
        y=dfc['growth_factor'],
        #line=dict(color='green', width=3),
        name='Fator de Crescimento',
        marker_color='lightslategray'
    ), row = 1, col = 1
)

fign.add_trace(
    go.Scatter(
        x=dfc['date'],
        y=dfc['growth_ratio'],
        #line=dict(color='green', width=3),
        name='Razão de Crescimento',
        marker_color='green'
    ), row = 2, col = 1
)

fign.add_trace(
    go.Scatter(
        x=dfc['date'],
        y=dfc['growth_rate'],
        #line=dict(color='green', width=3),
        marker_color='black',
        name='Taxa de crescimento'
    ), row = 3, col = 1
)

fign.add_trace(
    go.Scatter(
        x=dfc['date'],
        y=dfc['2derivative'],
        #line=dict(color='green', width=3),
        marker_color='crimson',
        name='2ª Derivada'
    ), row = 4, col = 1
)

fign.update_layout(hovermode = 'x',
                   title_text = 'Métricas de crescimento ao longo dos dias',
                   legend=dict(orientation = 'h', yanchor="bottom",y=1.01,xanchor="right",x=1))
fign.update_xaxes(title_text='Dia', row=4, col=1)
#fign.update_yaxes(title_text='Quantidade')
fign.show()

In [ ]:
path2 = '../input/corona-virus-brazil/brazil_covid19_cities.csv'
df2 = pd.read_csv(path2, encoding = 'utf-8')

df2['mortality'] = df2['deaths']*100/df2['cases']
df2['mortality'] = df2['mortality'].fillna(0)

df2.tail()

* **Extrator de valores para cada estado**

In [ ]:
class Extractor1:
    def __init__(self):
        pass
    
    def city(self, df, uf):
        dfa = df.loc[df['name'] == uf]
        df_city = dfa.groupby(['date'], as_index=False)[['cases','deaths']].sum()

        df_city['mortality'] = df_city['deaths']*100/df_city['cases']
        df_city['mortality'] = df_city['mortality'].fillna(0)
        
        df_city['new_cases'] = df_city.cases.diff()
        df_city['new_cases'] = df_city['new_cases'].fillna(0)
        df_city['new_deaths'] = df_city.deaths.diff()
        df_city['new_deaths'] = df_city['new_deaths'].fillna(0)
        
        return df_city
    
    def state(self, df, uf):
        dfa = df.loc[df['state'] == uf]
        df_state = dfa.groupby(['date'], as_index=False)[['cases','deaths']].sum()

        df_state['mortality'] = df_state['deaths']*100/df_state['cases']
        df_state['mortality'] = df_state['mortality'].fillna(0)
        
        df_state['new_cases'] = df_state.cases.diff()
        df_state['new_cases'] = df_state['new_cases'].fillna(0)
        df_state['new_deaths'] = df_state.deaths.diff()
        df_state['new_deaths'] = df_state['new_deaths'].fillna(0)
        
        return df_state

In [ ]:
df_state_per_day = df2.groupby(['date', 'state'], as_index=False)[['cases','deaths']].sum()
df_state_per_day['mortality'] = (df_state_per_day['deaths']*100)/df_state_per_day['cases']
df_state_per_day['mortality'] = df_state_per_day['mortality'].fillna(0)
df_state_per_day.tail()

dfb = df_state_per_day.groupby(['state'], as_index = False)['mortality'].mean()
#dfb.head()

dff = df2.groupby(['state','name'], as_index = False)[['cases', 'deaths']].max()
dff = dff.groupby(['state'], as_index = False)[['cases', 'deaths']].sum()

df_states = df_pop2.loc[:, ['UF', 'capital', 'população', 'IDH']]

selected_columns = dff[['cases','deaths']]
df_states[['cases', 'deaths']] = selected_columns.copy()

df_states['current_mortality'] = df_states['deaths']*100/df_states['cases']
df_states['cases100k'] = df_states['cases']*(100000/df_states['população'])
df_states['deaths100k'] = df_states['deaths']*(100000/df_states['população'])
df_states['avgmortality'] = dfb['mortality']
df_states.head(27)

In [ ]:
fig_states = go.Figure()

fig_states.add_trace(
    go.Bar(
        x=df_states['UF'],
        y=df_states['cases100k'],
        #line=dict(width=3),
        name='Casos',
        marker_color='gray',
        text = df_states['cases100k'],
        texttemplate='%{text:.2f}',
        textposition = 'inside'
    ))

fig_states.add_trace(
    go.Bar(
        x=df_states['UF'],
        y=df_states['deaths100k'],
        #line=dict(width=3),
        name='Mortes',
        marker_color='#d62728',
        #text = df_states['deaths100k'],
        #texttemplate='%{text:.0f}',
        #textposition = 'auto'
    ))


fig_states.update_layout(hovermode = 'x',
                         barmode = 'overlay',
                         title_text = 'Número de casos e de Mortes por 100.000 habitantes nos estados',
                         legend = dict(yanchor="top",y=0.97,xanchor="left",x=0.01))
                        
fig_states.update_layout(uniformtext_minsize=8, uniformtext_mode='hide')
fig_states.update_xaxes(title_text='Estado')
fig_states.update_yaxes(title_text='Quantidade')
fig_states.show()

In [ ]:
brazil_avg = df['mortality'].mean()

fig_states = go.Figure()

fig_states.add_trace(
    go.Bar(
        x=df_states['UF'],
        y=df_states['avgmortality'],
        name='Mortalidade',
        marker_color='#d62728',
        text = df_states['avgmortality'],
        texttemplate='%{text:.2f}',
        textposition = 'outside'))

fig_states.add_trace(
    go.Scatter(
         x=df_states['UF'],
         y=[brazil_avg]*len(df_states),
         line=dict(color='blue', width=0.7),
         #marker_color='blue',
         name='Média Nacional: %.2f' % brazil_avg))


fig_states.update_layout(barmode = 'stack',
                         title_text = 'Mortalidade média nos estados brasileiros',
                         legend = dict(yanchor="top",y=0.97,xanchor="left",x=0.01)
                        )
fig_states.update_xaxes(title_text='Estado')
fig_states.update_yaxes(title_text='Mortalidade média (%)')
fig_states.show()

In [ ]:
state_list = list(df_state_per_day['state'].unique())

figx = plt.figure(figsize= (13, 30))
plt.suptitle('Número de casos e de mortes confirmadas nos estados', fontsize = 20,y=1.01)

for i in range(len(state_list)):
    dfa = df_state_per_day.loc[df_state_per_day['state'] == state_list[i]]
    dates = list(pd.to_datetime(dfa['date']))
    
    ax = figx.add_subplot(14,2,i+1)
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%d-%b'))
    ax.bar(dates,dfa['cases'],color = 'green',alpha = 0.6,label = 'Casos');
    ax.bar(dates,dfa['deaths'],color='red', alpha = 0.6,label = 'Mortes');
    
    plt.title(state_list[i] + ' - Mortalidade média: %.4s' % (np.mean(dfa['mortality'])))
    handles, labels = ax.get_legend_handles_labels() #return handles and labels for legend
    figx.legend(handles, labels, loc="upper left")#, bbox_to_anchor=(1, 0.5))
    plt.subplots_adjust(hspace=0.5) # the amount of height reserved for space between subplots
    
    
figx.tight_layout()
figx.show()

In [ ]:
figabc = px.scatter(df_states, x='IDH', y='avgmortality', hover_data=['UF'], trendline="ols")
figabc.show()

corr1 = df_states['IDH'].corr(df_states['avgmortality'])
corr1

Extraindo valores para cada Cidade

In [ ]:
extractor = Extractor1()
df_df = extractor.city(df2, 'Brasília')
#df_df.head()

fig2 = make_subplots(rows=1, cols=2, shared_yaxes=True)

fig2.add_trace(
    go.Bar(
        x=df_df['date'],
        y=df_df['cases'],
        #line=dict(color='blue', width=3),
        name='Casos',
        marker_color='dodgerblue'))

fig2.add_trace(
    go.Bar(
        x=df_df['date'],
        y=df_df['deaths'],
        #line=dict(color='blue', width=3),
        name='Mortes',
        marker_color='red'))

fig2.add_trace(
    go.Bar(
        x=['Casos', 'Mortes'],
        y=[df_df['cases'].max(), df_df['deaths'].max()],
        #line=dict(color='green', width=3),
        marker_color=['dodgerblue','red'],
        showlegend=False,
        text = [df_df['cases'].max(), df_df['deaths'].max()],
        texttemplate='%{text:.f}',
        textposition = 'outside'), row=1, col=2)

fig2.update_layout(hovermode = 'x',
                  barmode = 'overlay',
                  title_text = 'Total de casos e casos confirmados por dia',
                  legend = dict(orientation = 'h', yanchor="bottom",y=1.01,xanchor="right",x=1))


fig2.update_xaxes(title_text="Dia", row=1, col=1)
fig2.update_xaxes(title_text="Total de casos e de mortes", row=1, col=2)                
fig2.update_yaxes(title_text='Quantidade',row=1, col=1)

fig2.show()

In [ ]:
fig3 = make_subplots(rows=1, cols=2, shared_yaxes=True)

fig3.add_trace(
    go.Bar(x=df['date'], 
           y=(df['mortality']),
           #line=dict(color='green', width=3),
           name='Brasil',
           marker_color = 'gray'
          )
)

fig3.add_trace(
    go.Bar(x=df_df['date'],
           y=df_df['mortality'],
           #line=dict(color='blue', width=3),
           name='Brasília',
           marker_color = 'green'
          )
)

fig3.add_trace(
    go.Bar(
        x=['Brasil', 'Brasilia'],
        y=[df['mortality'].mean(), df_df['mortality'].mean()],
        #line=dict(color='green', width=3),
        marker_color=['gray','green'],
        showlegend=False,
        text = [df['mortality'].mean(), df_df['mortality'].mean()],
        texttemplate='%{text:.2f}',
        textposition = 'outside'), row=1, col=2)

fig3.update_layout(hovermode = 'x',
                  barmode = 'overlay',
                  title_text = 'Taxa de mortalidade por dia e mortalidade média no Brasil e em Brasília',
                  legend = dict(orientation = 'h', yanchor="bottom",y=1.01,xanchor="right",x=1))


fig3.update_xaxes(title_text="Dia", row=1, col=1)
fig3.update_xaxes(title_text="Mortalidade média", row=1, col=2)                
fig3.update_yaxes(title_text='Mortalidade (%)',row=1, col=1)

fig3.show()

In [ ]:
path3 = '../input/corona-virus-brazil/brazil_cities_coordinates.csv'
df3 = pd.read_csv(path3, encoding = 'utf-8')

dfa = df3.loc[df3['capital'] == True]
dfa = dfa.reset_index(drop = True)
dfa = dfa.drop(['state_code', 'city_code', 'capital'], axis = 1)

capitals_list = list(dfa['city_name'])
lat = list(dfa['lat'])
long = list(dfa['long'])
#print(latt, longg)

In [ ]:
cmortality = []
tcases = []
tdeaths = []
amortality = []
Extractor = Extractor1()
for x in capitals_list:
    dff = Extractor.city(df2, x)
    a = np.max(dff['cases'])
    b = np.max(dff['deaths'])
    c = b*100/a
    mmean = np.mean(dff['mortality'])
    cmortality.append(mmean)
    tcases.append(a)
    tdeaths.append(b)
    amortality.append(c)
    

In [ ]:
#[capitals_list, tcases, tdeaths, amortality, cmortality]

In [ ]:
gini_list = [0.6341, 0.6284, 0.6106, 0.5936, 0.6370, 0.5815, 0.6008, 0.5652, 0.5474, 
             0.6267, 0.5908, 0.6287, 0.6037, 0.6378, 0.6334, 0.6217, 0.5529, 0.6144, 
             0.5745, 0.6894, 0.5123, 0.6391, 0.6449, 0.6266, 0.6453, 0.6171, 0.6124]

df_cor = pd.DataFrame(zip(capitals_list, tcases, tdeaths, amortality, cmortality, gini_list, lat, long),
                      columns=['city', 'tcases', 'tdeaths', 'cmortality', 'avgmortality', 'gini', 'lat', 'long'])
df_cor.sort_values(by=['tcases'], inplace=True)
df_cor = df_cor.reset_index(drop = True)
#print(len(df_cor['city']))
df_cor.tail()

In [ ]:
fig = go.Figure()
fig.add_trace(
    go.Bar(
        x = df_cor.iloc[-10:]['tcases'],
        y = df_cor.iloc[-10:]['city'],
        name = 'Casos',
        orientation = 'h'
    )
)

fig.add_trace(
    go.Bar(
        x = df_cor.iloc[-10:]['tdeaths'],
        y = df_cor.iloc[-10:]['city'],
        name = 'Mortes',
        orientation = 'h'
    )
)


fig.update_layout(hovermode = 'y',
                  barmode = 'overlay',
                  title_text = 'Dez capitais com maior número de casos',
                  legend = dict(orientation = 'h', yanchor="bottom",y=1.01,xanchor="right",x=1))


fig.update_xaxes(title_text = 'Quantidade')                
fig.update_yaxes(title_text = 'Capital')

fig.show()

In [ ]:
capitals_list2 = list(df_cor['city'].values[-5:])
fig_caps = go.Figure()

for caps in capitals_list2:
    df_city = Extractor.city(df2, caps)
    fig_caps.add_trace(
        go.Scatter(
            x=df_city['date'],
            y=df_city['mortality'],
            name=caps))

fig_caps.add_trace(
    go.Scatter(x=df['date'],
               y=df['mortality'],
               line=dict(color='black', width=3),
               name='Brasil'))
    
fig_caps.update_layout(title_text = 'Mortalidade por dia nas 5 capitais com mais casos confirmados',
                       legend = dict(yanchor="top",y=0.97,xanchor="left",x=0.01))
fig_caps.update_xaxes(title_text='Dia')
fig_caps.update_yaxes(title_text='Mortalidade (%)')
fig_caps.show()

In [ ]:
capitals_list3 = list(df_cor['city'].values[-10:])
print(capitals_list3)

figx = plt.figure(figsize= (14,14))
plt.suptitle('Quantidade de casos e número de mortes confirmadas nas 10 capitais mais afetadas',fontsize = 20,y=1.03)
k=0
for i in range(1,11):
    dfa = Extractor.city(df2, capitals_list3[k])
    dates = list(pd.to_datetime(dfa['date']))
    
    ax = figx.add_subplot(5,2,i)
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%d-%b'))
    ax.bar(dates,dfa['cases'],color = 'green',alpha = 0.6,label = 'Casos');
    ax.bar(dates,dfa['deaths'],color='red', alpha = 0.6,label = 'Mortes');
    
    plt.title(capitals_list3[k]+' - Mortalidade média: %.4s' % (np.mean(dfa['mortality'])))
    handles, labels = ax.get_legend_handles_labels() #return handles and labels for legend
    figx.legend(handles, labels, loc='upper left')
    plt.subplots_adjust(hspace=0.5) # the amount of height reserved for space between subplots
    
    k+=1

figx.tight_layout()
figx.show()


In [ ]:
fig = px.scatter(df_cor, x="gini", y="avgmortality", hover_data=['city'], trendline="ols")
fig.show()

corr = df_cor['gini'].corr(df_cor['avgmortality'])
corr


https://www.kaggle.com/nitishabharathi/the-story-of-covid-19-in-india-eda-and-prediction

In [ ]:
br_estados = '../input/brazil-geojson/brazil_geo.json'
geo_json_data = json.load(open(br_estados))

In [ ]:
mapa = folium.Map(
    width=600, height=400,
    location=[-15.77972, -47.92972], #centralizado no DF
    zoom_start=3
)
folium.GeoJson(geo_json_data).add_to(mapa)

for i in range(0,len(df_cor)):
   folium.Circle(
      location=[df_cor.iloc[i]['lat'], df_cor.iloc[i]['long']],
      popup=df_cor.iloc[i]['city'],
      radius=float(df_cor.iloc[i]['tcases'])*1,
      color='crimson',
      fill=True,
      fill_color='crimson'
   ).add_to(mapa)

mapa

https://www.kaggle.com/thiagobodruk/covid-19-brazil-analysis
atualizar mortalidade por 100.000 habs
https://www.kaggle.com/dferhadi/covid-19-predictions-growth-factor-and-calculus
https://www.kaggle.com/dferhadi/logistic-curve-fit-parameter-tuning
otimizar construção dos dataframes https://www.kaggle.com/thiagobodruk/covid-19-brazil-analysis